In [382]:
import time
import psycopg2
import pandas as pd

In [383]:
#to close other Connections before droping the database if it exists
def terminate_connections(dbname):
    try:
        conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=123456")
        conn.set_session(autocommit=True)
        cur = conn.cursor()

        # Query to find active connections to the specified database
        query = f"SELECT pg_terminate_backend(pid) FROM pg_stat_activity WHERE datname = '{dbname}'"
        cur.execute(query)

        print(f"Terminated connections to '{dbname}' database.")

        conn.close()
    except psycopg2.Error as e:
        print(f"Error terminating connections to '{dbname}' database")
        print(e)

In [384]:
def create_database():
    try: 
        # Connect to the default 'postgres' database
        conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=123456")
        print("Connected successfully to the default database.")
    except psycopg2.Error as e: 
        print("Error: Could not make connection to the Postgres database")
        print(e)
        return None, None
    
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # Terminate connections to 'accounts' database if it exists
    terminate_connections('accounts')
    
    # Drop 'accounts' database if it exists
    try:
        cur.execute("DROP DATABASE IF EXISTS accounts")
        print("Dropped 'accounts' database successfully.")
    except psycopg2.Error as e:
        print("Error dropping database")
        print(e)
        conn.close()
        return None, None
    
    # Create 'accounts' database
    try:
        cur.execute("CREATE DATABASE accounts")
        print("Created 'accounts' database successfully.")
    except psycopg2.Error as e:
        print("Error creating database")
        print(e)
        conn.close()
        return None, None
    
    # Close connection to the default database
    conn.close()
    
    # Connect to the newly created 'accounts' database 
    try:
        conn = psycopg2.connect("host=localhost dbname=accounts user=postgres password=123456")
        print("Connected successfully to the 'accounts' database.")
    except psycopg2.Error as e:
        print("Error: Could not connect to the newly created 'accounts' database")
        print(e)
        return None, None
    
    cur = conn.cursor()
    
    return cur, conn

# Example usage:
# cur, conn = create_database()
# # Use 'cur' and 'conn' for further database operations


In [385]:
def drop_tables(cur , conn):
    for query in drop_table_queries:
        cur.execute(query) 
        conn.commit()
        
        

In [386]:
def create_tables(cur , conn):
    for query in create_table_queries:
        cur.execute(query) 
        conn.commit()
        
        

In [387]:
AcountsCountry = pd.read_csv("Wealth-AccountsCountry.csv")
AcountsCountry.head(2)

,Code,Long Name,Income Group,Region,Lending category,Other groups,Currency Unit,Latest population census,Latest household survey,Special Notes,...,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data,2-alpha code,WB-2 code,Table Name,Short Name
0,ALB,Republic of Albania,Upper middle income,Europe & Central Asia,IBRD,NaN,Albanian lek,2020 (expected),"Demographic and Health Survey, 2017/18",NaN,...,Living Standards Measurement Study Survey (LSM...,Yes,2012,2013.0,2018.0,2006.0,AL,AL,Albania,Albania
1,ARG,Argentine Republic,Upper middle income,Latin America & Caribbean,IBRD,NaN,Argentine peso,2020 (expected),"Multiple Indicator Cluster Survey, 2019/20",NaN,...,"Integrated household survey (IHS), 2016",Yes,2008,2002.0,2018.0,2011.0,AR,AR,Argentina,Argentina


In [388]:
AcountsCountry_clean = AcountsCountry[["Code" , 'Short Name' , 'Table Name' , 'Long Name' , 'Currency Unit']]
AcountsCountry_clean

,Code,Short Name,Table Name,Long Name,Currency Unit
0,ALB,Albania,Albania,Republic of Albania,Albanian lek
1,ARG,Argentina,Argentina,Argentine Republic,Argentine peso
2,ARM,Armenia,Armenia,Republic of Armenia,Armenian dram
3,AUS,Australia,Australia,Commonwealth of Australia,Australian dollar
4,AUT,Austria,Austria,Republic of Austria,Euro
...,...,...,...,...,...
7592,NaN,NaN,NaN,NaN,NaN
7593,NaN,NaN,NaN,NaN,NaN
7594,NaN,NaN,NaN,NaN,NaN
7595,NaN,NaN,NaN,Data from database: Wealth Accounts,NaN


In [389]:
AccountsData = pd.read_csv("Wealth-AccountData.csv")
AccountsData.head(2)

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.34E+10,3.71E+10,3.88E+10,4.22E+10,4.36E+10,...,6.610000e+10,6.810000e+10,6.850000e+10,7.080000e+10,7.160000e+10,7.250000e+10,7.370000e+10,7.57E+10,7.80E+10,8.12E+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,13708.99,11794.1,12399.48,13573.26,14118.13,...,2.258200e+04,2.336286e+04,2.359058e+04,2.439397e+04,2.473162e+04,2.509002e+04,2.559371e+04,26326.26,27160.86,28333.68


In [390]:
AccountsData.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995 [YR1995]', '1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]',
       '1999 [YR1999]', '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]',
       '2003 [YR2003]', '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]',
       '2007 [YR2007]', '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]',
       '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]',
       '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]'],
      dtype='object')

In [391]:
AccountsData = AccountsData[['Country Name', 'Country Code', 'Series Name', 'Series Code','1995 [YR1995]','2000 [YR2000]', '2005 [YR2005]','2010 [YR2010]','2014 [YR2014]']]

AccountsData.head(5)

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],2000 [YR2000],2005 [YR2005],2010 [YR2010],2014 [YR2014]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.36E+10,5.440000e+10,6.810000e+10,7.250000e+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,14118.13,1.807230e+04,2.336286e+04,2.509002e+04
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.858,3424.007,2.222115e+03,2.889150e+03,2.957420e+03
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.931,6872.019,1.168935e+04,1.578992e+04,1.699138e+04
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.006,4695.071,2.886736e+03,3.613606e+03,3.719608e+03


In [392]:
new_names = {'1995 [YR1995]':"1995",'2000 [YR2000]':"2000", '2005 [YR2005]':"2005",'2010 [YR2010]':"2010",'2014 [YR2014]':"2014"}
AccountsData.rename(columns=new_names, inplace=True)
AccountsData.head(2)

,Country Name,Country Code,Series Name,Series Code,1995,2000,2005,2010,2014
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.36E+10,5.440000e+10,6.810000e+10,7.250000e+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,14118.13,1.807230e+04,2.336286e+04,2.509002e+04


In [393]:
AccountsSeries = pd.read_csv("Wealth-AccountSeries.csv")
AccountsSeries.head(2)

,Code,Indicator Name,Long definition,Source,Topic,Unit of measure,Periodicity,Reference period,Statistical concept and methodology,Previous Indicator Code,Previous Indicator Name
0,NW.HCA.TO,Human capital (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
1,NW.HCA.PC,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN


In [394]:
AccountsSeries.columns


Index(['Code', 'Indicator Name', 'Long definition', 'Source', 'Topic',
       'Unit of measure', 'Periodicity', 'Reference period',
       'Statistical concept and methodology', 'Previous Indicator Code',
       'Previous Indicator Name'],
      dtype='object')

In [395]:
AccountsSeries =AccountsSeries[['Code' , 'Topic' , 'Indicator Name' , 'Long definition']]
AccountsSeries.head(2)

,Code,Topic,Indicator Name,Long definition
0,NW.HCA.TO,Human capital,Human capital (constant 2018 US$),Human capital is computed as the present value...
1,NW.HCA.PC,Human capital,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...


In [396]:
cur , conn = create_database()

Connected successfully to the default database.
Terminated connections to 'accounts' database.
Dropped 'accounts' database successfully.
Created 'accounts' database successfully.
Connected successfully to the 'accounts' database.


In [397]:
songplay_table_create  = ("""CREATE TABLE IF NOT EXISTS accountscountry(
country_code VARCHAR ,
short_name VARCHAR,
table_name VARCHAR,
long_name VARCHAR,
currency_unit VARCHAR
)""")



In [398]:
cur.execute(songplay_table_create)
conn.commit()

In [399]:
accounts_data_table_create = ("""CREATE TABLE IF NOT EXISTS accountsdata(
country_name VARCHAR,
country_code VARCHAR,
indicator_name VARCHAR,
indicator_code VARCHAR,
year_1995 VARCHAR,
year_2000 VARCHAR,
year_2005 VARCHAR,
year_2010 VARCHAR,
year_2014 VARCHAR
)""")
cur.execute(accounts_data_table_create)
conn.commit()

In [400]:
accountseries_data_table_create=("""CREATE TABLE IF NOT EXISTS accountseries(
series_code VARCHAR,
topic VARCHAR,
indicator_name VARCHAR,
short_definition VARCHAR
)""")
cur.execute(accountseries_data_table_create)
conn.commit()

In [401]:
cur.execute("truncate table accountscountry")
conn.commit()

In [402]:
accounts_country_table_insert = ("""INSERT INTO accountscountry(
country_code,
short_name,
table_name,
long_name,
currency_unit)
VALUES (%s, %s, %s, %s, %s)
""")


In [403]:
for  i , row in AcountsCountry_clean.iterrows():
    cur.execute(accounts_country_table_insert , list(row))
conn.commit()    

In [404]:
accounts_data_table_insert = ("""INSERT INTO accountsdata(
country_name,
country_code,
indicator_name,
indicator_code,
year_1995,
year_2000,
year_2005,
year_2010,
year_2014)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
""")


In [405]:
for i, row in AccountsData.iterrows():
    cur.execute(accounts_data_table_insert, list(row))
conn.commit()

In [406]:
accountseries_data_table_insert = ("""INSERT INTO accountseries(
series_code,
topic,
indicator_name,
short_definition)
VALUES (%s, %s, %s, %s)
""")


In [407]:
for i, row in AccountsSeries.iterrows():
    cur.execute(accountseries_data_table_insert, list(row))
conn.commit()